# 02 - Data Preprocessing

**Purpose**: Clean data for next notebook.

In [1]:
import sys; sys.path.append('..')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import joblib

In [2]:
# Paths
DATA_RAW_PATH = '../data/raw/cancer_endometri.csv'
DATA_PROCESSED_PATH = '../data/processed/'
MODELS_PATH = '../models/'

# Target
TARGET = 'recidiva_exitus'

# Columnes a eliminar (leakage + soroll), de l'EDA
LEAKAGE_COLUMNS = [
    'recid_super_1', 'recidiva', 'fecha_de_recidi', 'f_muerte', 'causa_muerte',
    'tto_recidiva', 'Tt_recidiva_qx', 'otro_ttIQ_recid',
    'loc_recidiva_r01', 'loc_recidiva_r02', 'loc_recidiva_r03',
    'loc_recidiva_r04', 'loc_recidiva_r05', 'loc_recidiva_r06',
    'numero_de_recid', 'num_recidiva', 'dx_recidiva', 'libre_enferm',
    'est_pcte', 'estado', 'visita_control', 'Ultima_fecha',
    'diferencia_dias_reci_exit', 'despues_diag',
    'codigo_participante', 'usuario_reg1', 'f_diag', 'FN', 'fecha_qx', 'f_1v',
    'f_tto_NA', 'comentarios', 'otras_especifi', 'otra_histo', 'histo_otros',
    'ap_comentarios', 'inicio_qmt', 'fecha_final_qmt', 'tt_o_f_ini', 'tt_o_f_fin',
    'ini_bqt_rt', 'final_bqt_rt', 'ap_gPelv_loc', 'Tributaria_a_Radioterapia',
    'otros_tt', 'qt', 'bqt', 'moti_no_RT', 'rdt', 'estadificacion_',
    'dias_de_ingreso', 'asa', 'tiempo_qx', 'centro_tratPrima', 'motivonolaparos',
    'trazador_utiliz', 'tabla_de_estadi', 'tabla_de_riesgo',
    'compl_precoc_r01', 'compl_precoc_r02', 'compl_precoc_r03',
    'compl_precoc_r04', 'compl_precoc_r05', 'compl_precoc_r06',
    'compl_precoc_r07', 'compl_precoc_r08', 'compl_precoc_r09',
    'compl_precoc_r10', 'compl_precoc_r11', 'compl_precoc_r12',
    'compl_precoc_r13', 'compl_precoc_r14',
    'comp_intraop_r01', 'comp_intraop_r02', 'comp_intraop_r03',
    'comp_intraop_r04', 'comp_intraop_r05', 'comp_intraop_r06',
    'comp_intraop_r07', 'otras', 'comp_claviendin_mes',
    'reintervencion', 'reintervencion_motivo', 'tiempo_transcur',
    'perdida_hematic', 'perdida_hem_cc', 'n_resec_Intes', 'oment_Avan',
    'ciclos_tto_NAdj', 'transf_GRC',
    'estudio_genetico_r01', 'estudio_genetico_r02', 'estudio_genetico_r03',
    'estudio_genetico_r04', 'estudio_genetico_r05', 'estudio_genetico_r06',
    'ultraestidaije_GC', 'inf_param_vag',
    'p53_molecular', 'p53_ihq', 'mut_pole', 'msh2', 'msh6', 'pms2', 'mlh1',
    'Reseccion_macroscopica_complet', 'Tratamiento_RT', 'Tratamiento_sistemico',
    'presntado_cTG', 
]

# Prefixos a excloure
PREFIXES_TO_EXCLUDE = ['tec_', 'gc_', 'Motivo_de_conversion_']

In [3]:
# Carregar dades
df = pd.read_csv(DATA_RAW_PATH)

# Netejar noms de columnes (eliminar espais)
df.columns = df.columns.str.strip()

print(f"Dades carregades: {df.shape[0]} files x {df.shape[1]} columnes")

# Filtrar columnes (eliminar leakage i soroll)
columns_to_keep = [
    col for col in df.columns 
    if col not in LEAKAGE_COLUMNS 
    and not any(col.startswith(prefix) for prefix in PREFIXES_TO_EXCLUDE)
]

df = df[columns_to_keep].copy()

print(f"Després de filtrar columnes: {df.shape[0]} files x {df.shape[1]} columnes")
print(f"\nColumnes disponibles: {list(df.columns)}")

Dades carregades: 163 files x 189 columnes
Després de filtrar columnes: 163 files x 56 columnes

Columnes disponibles: ['recidiva_exitus', 'edad', 'imc', 'tipo_histologico', 'Grado', 'valor_de_ca125', 'ecotv_infiltsub', 'ecotv_infiltobj', 'metasta_distan', 'grupo_riesgo', 'estadiaje_pre_i', 'hsp_trat_primario', 'tto_NA', 'tto_1_quirugico', 'inten_tto', 'abordajeqx', 'conver_laparo', 'Tec_histerec', 'Anexectomia', 'omentectomia', 'Perforacion_uterina', 'Movilizador_uterino', 'tc_gc', 'histe_avanz', 'histo_defin', 'grado_histologi', 'tamano_tumoral', 'infilt_estr_cervix', 'infiltracion_mi', 'tx_anexial', 'tx_sincronico', 'afectacion_linf', 'afectacion_omen', 'AP_centinela_pelvico', 'n_total_GC', 'n_GC_Afect', 'AP_ganPelv', 'n_total_ganCent', 'n_gangP_afec', 'AP_glanPaor', 'Local_Gan_Paor', 'n_total_ganPaor_infra', 'n_ganPaor_InfrM_afec', 'n_total_ganPaor_supr', 'n_ganPaor_Sup_afec', 'ap_gPaor_total', 'ap_gPor_afect_tot', 'recep_est_porcent', 'rece_de_Ppor', 'beta_cateninap', 'FIGO2023', 

In [4]:
# Eliminar files on recidiva_exitus és NA o 2 (Desconegut)
df = df[df[TARGET].notna()]
df = df[df[TARGET] != 2]

# Convertir target a int
df[TARGET] = df[TARGET].astype(int)

print(f"Després de filtrar target: {df.shape[0]} files x {df.shape[1]} columnes")
print(f"\nDistribució del target:\n{df[TARGET].value_counts()}")

Després de filtrar target: 154 files x 56 columnes

Distribució del target:
recidiva_exitus
0    120
1     34
Name: count, dtype: int64


In [5]:
# Veure quines columnes tenen NAs i quin percentatge
na_percent = (df.isna().sum() / len(df) * 100).sort_values(ascending=False)
cols_with_na = na_percent[na_percent > 0]
print(f"Columnes amb NAs ({len(cols_with_na)}):")
for col, pct in cols_with_na.items():
    print(f"  {col}: {pct:.1f}%")

Columnes amb NAs (50):
  histe_avanz: 90.9%
  valor_de_ca125: 86.4%
  Tratamiento_sistemico_realizad: 81.2%
  AP_glanPaor: 79.2%
  rt_dosis: 79.2%
  Local_Gan_Paor: 79.2%
  bt_realPac: 76.0%
  AP_ganPelv: 64.9%
  n_doisis_rt: 51.3%
  rece_de_Ppor: 44.8%
  recep_est_porcent: 43.5%
  n_ganPaor_Sup_afec: 39.0%
  n_ganPaor_InfrM_afec: 38.3%
  n_total_ganPaor_supr: 35.1%
  n_total_ganPaor_infra: 34.4%
  ap_gPor_afect_tot: 29.9%
  ap_gPaor_total: 26.6%
  n_gangP_afec: 25.3%
  afectacion_omen: 22.7%
  n_total_ganCent: 20.8%
  conver_laparo: 19.5%
  tamano_tumoral: 18.2%
  n_GC_Afect: 18.2%
  omentectomia: 17.5%
  AP_centinela_pelvico: 17.5%
  Movilizador_uterino: 17.5%
  Perforacion_uterina: 16.2%
  n_total_GC: 16.2%
  Anexectomia: 15.6%
  Tec_histerec: 15.6%
  tc_gc: 14.3%
  FIGO2023: 11.7%
  grupo_de_riesgo_definitivo: 10.4%
  afectacion_linf: 10.4%
  tx_anexial: 9.7%
  grado_histologi: 9.7%
  abordajeqx: 8.4%
  infiltracion_mi: 8.4%
  beta_cateninap: 7.1%
  infilt_estr_cervix: 6.5%
  tx_si

In [6]:
# histe_avanz (90.9% NA)
def impute_histe_avanz(val):
    if val == 0:
        return 0
    elif pd.isna(val) or val != 2:
        return 1
    else:
        return val  # Manté el 2 si existeix

df['histe_avanz'] = df['histe_avanz'].apply(impute_histe_avanz)

print("Resultat imputació histe_avanz:")
print(df['histe_avanz'].value_counts(dropna=False))
print(f"NAs restants: {df['histe_avanz'].isna().sum()}")

# valor_de_ca125 (86.4% NA)

# Llindar clínic: 35 U/mL
def categorize_ca125(val):
    if pd.isna(val):
        return "Desconocido"
    elif val < 35:
        return "Normal"
    else:
        return "Elevado"

df['valor_de_ca125'] = df['valor_de_ca125'].apply(categorize_ca125)

print("\nResultat categorització valor_de_ca125:")
print(df['valor_de_ca125'].value_counts(dropna=False))

Resultat imputació histe_avanz:
histe_avanz
1.0    140
0.0     13
2.0      1
Name: count, dtype: int64
NAs restants: 0

Resultat categorització valor_de_ca125:
valor_de_ca125
Desconocido    133
Normal          11
Elevado         10
Name: count, dtype: int64


In [7]:
# Variables base
# Grau histològic: fem servir la moda perquè és necessària per a altres imputacions
grado_mode = df['grado_histologi'].mode()[0]
df['grado_histologi'] = df['grado_histologi'].fillna(grado_mode)

# Variables de ganglis i radioteràpia
# -1 indica no valorat / desconegut
df['AP_glanPaor'] = df['AP_glanPaor'].fillna(-1)
df['AP_ganPelv'] = df['AP_ganPelv'].fillna(-1)
df['AP_centinela_pelvico'] = df['AP_centinela_pelvico'].fillna(-1)
df['rt_dosis'] = df['rt_dosis'].fillna(-1)

# Ganglis paraòrtics
df['Local_Gan_Paor'] = df['Local_Gan_Paor'].fillna(-1)

# Nombre total de ganglis paraòrtics
# 0 si no s’ha valorat, -1 si és desconegut
for col in ['n_total_ganPaor_supr', 'n_total_ganPaor_infra', 'ap_gPaor_total']:
    df[col] = df.apply(
        lambda r: 0 if pd.isna(r[col]) and r['AP_glanPaor'] == -1
        else (-1 if pd.isna(r[col]) else r[col]),
        axis=1
    )

# Ganglis paraòrtics afectats
# 0 si no hi ha afectació o no s’ha valorat, -1 si és desconegut
for col in ['n_ganPaor_Sup_afec', 'n_ganPaor_InfrM_afec', 'ap_gPor_afect_tot']:
    df[col] = df.apply(
        lambda r: 0 if pd.isna(r[col]) and r['AP_glanPaor'] in [-1, 0]
        else (-1 if pd.isna(r[col]) else r[col]),
        axis=1
    )

# Ganglis pèlvics afectats
df['n_gangP_afec'] = df.apply(
    lambda r: 0 if pd.isna(r['n_gangP_afec']) and r['AP_ganPelv'] in [-1, 0]
    else (-1 if pd.isna(r['n_gangP_afec']) else r['n_gangP_afec']),
    axis=1
)

# Gangli centinella
# Nombre total de ganglis centinella
for col in ['n_total_ganCent', 'n_total_GC']:
    df[col] = df.apply(
        lambda r: 0 if pd.isna(r[col]) and r['AP_centinela_pelvico'] == -1
        else (-1 if pd.isna(r[col]) else r[col]),
        axis=1
    )

# Ganglis centinella afectats
df['n_GC_Afect'] = df.apply(
    lambda r: 0 if pd.isna(r['n_GC_Afect']) and r['AP_centinela_pelvico'] in [-1, 0]
    else (-1 if pd.isna(r['n_GC_Afect']) else r['n_GC_Afect']),
    axis=1
)

# Radioteràpia
df['n_doisis_rt'] = df.apply(
    lambda r: 0 if pd.isna(r['n_doisis_rt']) and r['rt_dosis'] in [-1, 0]
    else (-1 if pd.isna(r['n_doisis_rt']) else r['n_doisis_rt']),
    axis=1
)

# Variables numèriques
# Receptors i mida tumoral: mediana segons el grau histològic
for col in ['recep_est_porcent', 'rece_de_Ppor', 'tamano_tumoral']:
    median_by_grade = df.groupby('grado_histologi')[col].median()
    df[col] = df.apply(
        lambda r: median_by_grade.get(r['grado_histologi'], df[col].median())
        if pd.isna(r[col]) else r[col],
        axis=1
    )
    df[col] = df[col].fillna(df[col].median())

# IMC: mediana global
df['imc'] = df['imc'].fillna(df['imc'].median())

# Variables quirúrgiques binàries
# Si no consta, s’assumeix que no
binary_zero_vars = [
    'afectacion_omen', 'conver_laparo', 'omentectomia',
    'Perforacion_uterina', 'afectacion_linf', 'tx_anexial',
    'infilt_estr_cervix', 'tx_sincronico', 'metasta_distan'
]

for var in binary_zero_vars:
    df[var] = df[var].fillna(0)

# Variables quirúrgiques categòriques
# S’imputa la moda
for var in ['Anexectomia', 'Tec_histerec', 'abordajeqx']:
    df[var] = df[var].fillna(df[var].mode()[0])

# Variables clíniques i patològiques
# També s’imputen amb la moda
mode_clinical_vars = [
    'FIGO2023', 'grupo_de_riesgo_definitivo', 'infiltracion_mi',
    'histo_defin', 'ecotv_infiltobj', 'ecotv_infiltsub',
    'estadiaje_pre_i', 'grupo_riesgo'
]

for var in mode_clinical_vars:
    if df[var].isna().sum() > 0:
        df[var] = df[var].fillna(df[var].mode()[0])

# Variables amb categoria desconegut (-1)
for var in ['Movilizador_uterino', 'tc_gc', 'bt_realPac']:
    df[var] = df[var].fillna(-1)

# Variables de tractament
df['Tratamiento_sistemico_realizad'] = df['Tratamiento_sistemico_realizad'].fillna(0)
df['inten_tto'] = df['inten_tto'].fillna(1)
df['tto_1_quirugico'] = df['tto_1_quirugico'].fillna(1)

# Marcadors moleculars
# 2 indica que no s’ha realitzat la prova
df['beta_cateninap'] = df['beta_cateninap'].fillna(2)


In [8]:
# Les 14 features definitives seleccionades pel model
FINAL_FEATURES = [
    'grupo_de_riesgo_definitivo',
    'afectacion_linf',
    'estadiaje_pre_i',
    'Tratamiento_sistemico_realizad',
    'grado_histologi',
    'infiltracion_mi',
    'imc',
    'FIGO2023',
    'recep_est_porcent',
    'rece_de_Ppor',
    'edad',
    'tto_1_quirugico',
    'histo_defin',
    'metasta_distan'
]

FINAL_COLUMNS = ['recidiva_exitus'] + FINAL_FEATURES

df = df[FINAL_COLUMNS].copy()

print(f"Features definitives seleccionades: {len(FINAL_FEATURES)}")
print(f"Columnes finals: {list(df.columns)}")

Features definitives seleccionades: 14
Columnes finals: ['recidiva_exitus', 'grupo_de_riesgo_definitivo', 'afectacion_linf', 'estadiaje_pre_i', 'Tratamiento_sistemico_realizad', 'grado_histologi', 'infiltracion_mi', 'imc', 'FIGO2023', 'recep_est_porcent', 'rece_de_Ppor', 'edad', 'tto_1_quirugico', 'histo_defin', 'metasta_distan']


In [9]:
import os

DATA_PROCESSED_PATH = '../data/processed/'
os.makedirs(DATA_PROCESSED_PATH, exist_ok=True)

df.to_csv(f'{DATA_PROCESSED_PATH}preprocessed_v1.csv', index=False)

print(f"   {df.shape[0]} files x {df.shape[1]} columnes")

   154 files x 15 columnes
